In [2]:
from langchain.chains import RetrievalQA
from langchain_community.embeddings import MlflowEmbeddings
from langchain_pinecone import PineconeVectorStore
from domino_data.vectordb import domino_pinecone3x_init_params, domino_pinecone3x_index_paramsfrom langchain import PromptTemplate
from langchain_community.chat_models import ChatMlflow

import csv
import os
from pinecone import Pinecone
import sys

from mlflow.deployments import get_deploy_client
import os

import warnings
warnings.filterwarnings('ignore')

In [3]:

client = get_deploy_client(os.environ['DOMINO_MLFLOW_DEPLOYMENTS'])

In [4]:
# initialize embedding
embeddings = MlflowEmbeddings(
    target_uri=os.environ["DOMINO_MLFLOW_DEPLOYMENTS"],
    endpoint="embedding-ada-002ja2"
)

In [5]:
#Domino Vector Data Source name
datasource_name = "mrag-fin-docs-ja"
# Load Domino Pinecone Data Source Configuration 
pc = Pinecone(**domino_pinecone3x_init_params(datasource_name))


# Load Pinecone Index
index_name = "mrag-fin-docs"
index = pc.Index(**domino_pinecone3x_index_params(datasource_name, index_name))
text_field = "text"  # switch back to normal index for langchain
vectorstore = PineconeVectorStore(  
    index, embeddings, text_field   # Using embedded data from Domino AI Gateway Endpoint
)

In [6]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.00361,
 'namespaces': {'': {'vector_count': 361}},
 'total_vector_count': 361}

In [7]:

prompt_template = """You are an AI assistant with expertise in financial analysis. You are given the following extracted parts and a question. 
Question: {question}
=========
{context}
=========
Answer in Markdown:
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["question", "context"])
#
chain_type_kwargs = {"prompt": PROMPT}

In [8]:
rag_llm = ChatMlflow(
        target_uri=os.environ["DOMINO_MLFLOW_DEPLOYMENTS"],
        endpoint="chat-gpt35turbo-ja", 
        temperature=0.0,
    )

In [9]:

qa_chain = RetrievalQA.from_chain_type(llm=rag_llm, # AI Gateway Endpoint
                                       chain_type="stuff",
                                       chain_type_kwargs={"prompt": PROMPT},
                                       retriever=vectorstore.as_retriever(search_kwargs={"k": 20}), # Domino Pinecone Data Source
                                       return_source_documents=True
                                      )

In [10]:
user_question = input("Please ask your financial question:")
result = qa_chain(user_question)

Please ask your financial question: What was the gross income amount and percentage as share of total revenues in FY23


In [11]:
result['result']

'To calculate the gross income amount and percentage as a share of total revenues in FY23, we need to use the information provided in the "CONSOLIDATED STATEMENTS OF OPERATIONS" section.\n\nFrom the "CONSOLIDATED STATEMENTS OF OPERATIONS" for the year ended September 30, 2023:\n- Total net sales: $383,285 million\n- Gross margin: $169,148 million\n\nGross income amount = Total net sales - Gross margin\nGross income amount = $383,285 million - $169,148 million\nGross income amount = $214,137 million\n\nGross income percentage as a share of total revenues = (Gross income amount / Total net sales) * 100\nGross income percentage = ($214,137 million / $383,285 million) * 100\nGross income percentage = 55.9%\n\nTherefore, in FY23, the gross income amount was $214,137 million and the gross income percentage as a share of total revenues was 55.9%.'

In [14]:
print(result)

{'query': 'How much revenue did the Americas generate in FY23?', 'result': 'To determine the revenue generated by the Americas in FY23, we can refer to the provided information from the segment operating performance section of the document. \n\nIn FY23, the net sales generated by the Americas segment were $162,560 million. This represented a 4% decrease from the previous year (FY22) when the net sales were $169,658 million. \n\nTherefore, the revenue generated by the Americas in FY23 was $162,560 million.', 'source_documents': [Document(page_content='Segment Operating Performance\nThe following table shows net sales by reportable segment for 2023, 2022 and 2021 (dollars in millions):\n2023 Change 2022 Change 2021\nNet sales by reportable segment:\nAmericas $ 162,560 (4)%$ 169,658 11 %$ 153,306 \nEurope 94,294 (1)% 95,118 7 % 89,307 \nGreater China 72,559 (2)% 74,200 9 % 68,366 \nJapan 24,257 (7)% 25,977 (9)% 28,482 \nRest of Asia Pacific 29,615 1 % 29,375 11 % 26,356 \nTotal net sales 